In [1]:
import os, sys
# for accessing src, stan, etc.
sys.path.append(os.path.abspath(os.path.join("../..")))

import bayesflow as bf
import numpy as np
from src.AmortizedMixture import *
from src.models.NormalHmm import *


import matplotlib.pyplot as plt

/Users/simonkucharsky/projects/bayesflow/amortized-mixture/.venv/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
model = NormalHmm(n_cls = 2, n_obs=[100, 100], separation=3.0)
model.prior_means = [0.0, 0.0, -1.5, 1.00]
model.prior_sds = [1.14,  1.14, 1.0, 0.65]

In [3]:
summary_net = bf.networks.SequenceNetwork(summary_dim=model.n_par*2)
amortized_posterior=bf.amortizers.AmortizedPosterior(
            inference_net=bf.networks.InvertibleNetwork(num_params=model.n_par, num_coupling_layers=10, coupling_design="spline"), 
            summary_net=summary_net,
            summary_loss_fun="MMD"
        )

In [4]:
trainer = bf.trainers.Trainer(amortizer=amortized_posterior, generative_model=model, checkpoint_path="checkpoints/posterior", configurator=lambda x: x)

INFO:root:Initialized empty loss history.
INFO:root:Initialized networks from scratch.
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [5]:
%time
h = trainer.train_online(epochs=50, iterations_per_epoch=1000, batch_size=32)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs


Training epoch 14:  32%|███▏      | 318/1000 [00:35<01:16,  8.91it/s, Epoch: 14, Iter: 318,Loss: 1.066,W.Decay: 0.048,Avg.Loss: 1.433,Avg.W.Decay: 0.048,LR: 4.17E-04]

In [7]:
f=bf.diagnostics.plot_losses(h)

NameError: name 'h' is not defined

In [ ]:
df = model(1000, context = {'n_obs': 100})

In [ ]:
z, _ = trainer.amortizer(df)

In [ ]:
f = bf.diagnostics.plot_latent_space_2d(z)

In [ ]:
prior_samples = df['parameters']
posterior_samples = trainer.amortizer.sample(df, n_samples=1000)

In [ ]:
f = bf.diagnostics.plot_sbc_ecdf(posterior_samples, prior_samples, difference=True)

In [ ]:
f = bf.diagnostics.plot_sbc_ecdf(posterior_samples, prior_samples)

In [ ]:
f = bf.diagnostics.plot_z_score_contraction(posterior_samples, prior_samples)

In [ ]:
f = bf.diagnostics.plot_recovery(posterior_samples, prior_samples)